# 01. Load image and depth map

In [52]:
import open3d as o3d
import cv2
import numpy as np

# read grayscale image
# Load the RGB image
rgb_image = cv2.imread("im0.png")  # Reads as BGR

# Convert to grayscale (single channel)
# gray_image = cv2.cvtColor(rgb_image, cv2.COLOR_BGR2GRAY)

# # Ensure the image is in 16-bit format
# if gray_image.dtype != np.uint16:
#     gray_image = (gray_image.astype(np.float32) / 255.0 * 65535).astype(np.uint16)

### CHESS EXAMPLE ###
#color_raw = o3d.io.read_image("im0.png")
#depth_raw = o3d.io.read_image("depth_map.png")

### CUBE RESIZED EXAMPLE ###
color_path = "../dataset/cube_resized.jpg"
depth_path = "cube_resized_depth_map.png"

color_raw = o3d.io.read_image(color_path)
depth_raw = o3d.io.read_image(depth_path)



rbgd = o3d.geometry.RGBDImage.create_from_color_and_depth(
    color_raw, depth_raw, convert_rgb_to_intensity=False
)

print(rbgd)



RGBDImage of size 
Color image : 548x548, with 3 channels.
Depth image : 548x548, with 1 channels.
Use numpy.asarray to access buffer data.


# 02. Set up camera

In [53]:
camera_intrinsic = o3d.camera.PinholeCameraIntrinsic(
    o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault
)

# 03. Create a pointcloud

In [54]:
# pcd, ind = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
pcd = o3d.geometry.PointCloud.create_from_rgbd_image(
    rbgd, camera_intrinsic
)

pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])

PointCloud with 299958 points.

# 04. Choose only points with certain density inside the voxel

In [55]:
voxel_size = 0.00001  # Adjust this value based on your data

# Compute voxel grid
voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd, voxel_size)
voxels = voxel_grid.get_voxels()
print(len(voxels))
#o3d.visualization.draw_geometries([voxel_grid])

# Create a mapping from grid index to point count
voxel_dict = {}
point_indices = np.floor(np.asarray(pcd.points) / voxel_size).astype(int)

for index in point_indices:
    voxel_index = tuple(index)
    voxel_dict[voxel_index] = voxel_dict.get(voxel_index, 0) + 1

# Define density threshold
min_points_per_voxel = 10  # Adjust as needed

# Filter points based on voxel density
filtered_points = [
    p for i, p in enumerate(np.asarray(pcd.points))
    if voxel_dict.get(tuple(point_indices[i]), 0) >= min_points_per_voxel
]

# Create new filtered point cloud
filtered_pcd = o3d.geometry.PointCloud()
filtered_pcd.points = o3d.utility.Vector3dVector(filtered_points)

# Visualize the filtered point cloud
#o3d.visualization.draw_geometries([filtered_pcd])

12254


# 05. Downsample

In [56]:
# Downsample and get keypoints
voxel_size = 0.0001 # Size of the boundarying box
pcd_down = filtered_pcd.voxel_down_sample(voxel_size=voxel_size)

# Visualize the downsampled point cloud
#o3d.visualization.draw_geometries([pcd_down])

print("Number of points after downsampling: ", len(pcd_down.points))

Number of points after downsampling:  158


# 06. Segmentation task

In [57]:
# Segment the largest plane using RANSAC
plane_model, inliers = pcd_down.segment_plane(
                                         distance_threshold=0.0001,
                                         ransac_n=10,
                                         num_iterations=1000)

# Extract plane and non-plane points
plane = pcd_down.select_by_index(inliers)
plane.paint_uniform_color([1, 0, 0])  # Color plane red
non_plane = pcd_down.select_by_index(inliers, invert=True)

# Visualize
#o3d.visualization.draw_geometries([non_plane])

print("Number of points after plane segmentation: ", len(non_plane.points))


Number of points after plane segmentation:  59


# 06.b Delete the bulb of points distant from the main bulb of points

Apply clustering and keep bigger bulb of points

In [58]:
import open3d as o3d
import numpy as np
from sklearn.cluster import KMeans


# Convert point cloud to numpy array
points = np.asarray(non_plane.points)

# Define the number of clusters
num_clusters = 2  # Adjust this value based on your needs

# Perform K-Means clustering
kmeans = KMeans(n_clusters=num_clusters, max_iter=100000, init='k-means++', n_init=10).fit(points)
labels = kmeans.labels_

# Assign colors to each cluster
colors = np.random.rand(num_clusters, 3)
non_plane.colors = o3d.utility.Vector3dVector(colors[labels])


# Identify the larger cluster
unique, counts = np.unique(labels, return_counts=True)
larger_cluster_label = unique[np.argmax(counts)]

# Select points that belong to the larger cluster
larger_cluster_indices = np.where(labels == larger_cluster_label)[0]
larger_cluster = non_plane.select_by_index(larger_cluster_indices)

# Visualize the larger cluster
o3d.visualization.draw_geometries([larger_cluster], height=800, width=800)

c:\Users\ewa.miazga\miniconda3\envs\open3d_env\lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


# 06.c Take points that are at certain distance to non plane points

In [59]:
import open3d as o3d
import numpy as np
from scipy.spatial import KDTree

cube = larger_cluster

# Convert point clouds to numpy arrays
pcd_points = np.asarray(pcd.points)
cube_points = np.asarray(cube.points)

# Build a KDTree using the original point cloud
kdtree = KDTree(pcd_points)

# Define the distance threshold
distance_threshold = 0.0001  # Adjust this value as needed

# Find points within the distance threshold from the non-plane points
indices = kdtree.query_ball_point(cube_points, r=distance_threshold)

# Flatten the list of indices and remove duplicates
indices = np.unique(np.concatenate(indices)).astype(int).tolist()

# Select the filtered points from the original point cloud
filtered_pcd = pcd.select_by_index(indices)

# Delete outliers
#filtered_pcd, ind = filtered_pcd.remove_statistical_outlier(nb_neighbors=100000, std_ratio=12.0)

# Visualize the filtered points
#o3d.visualization.draw_geometries([filtered_pcd], height=800, width=800)

(if the kmeans applied before doesnot work )

In [60]:
 
# Convert point cloud to numpy array
# points = np.asarray(filtered_pcd.points)

# # Define the number of clusters
# num_clusters = 2  # Adjust this value based on your needs

# # Perform K-Means clustering
# kmeans = KMeans(n_clusters=num_clusters, max_iter=10, init='k-means++').fit(points)
# labels = kmeans.labels_

# # Assign colors to each cluster
# colors = np.random.rand(num_clusters, 3)
# filtered_pcd.colors = o3d.utility.Vector3dVector(colors[labels])

# # Visualize the clustered point cloud
# o3d.visualization.draw_geometries([filtered_pcd], height=800, width=800)

# 07. Calculate the normals

In [61]:
filtered_pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

# Visualize
#o3d.visualization.draw_geometries([filtered_pcd], 
#                                   point_show_normal=True,
#                                   mesh_show_wireframe=True, 
#                                   mesh_show_back_face=True
#                                   )

In [62]:
print(f"Number of points in cube: {len(filtered_pcd.points)}")

Number of points in cube: 90777


# 08. Create a grid out of those points that are left

Not used

In [63]:
# Define the voxel size for the grid
voxel_size = 0.00001  # Adjust this value based on your data

# Create a voxel grid from the non-plane point cloud
voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(filtered_pcd, voxel_size)


# Visualize the voxel grid
#o3d.visualization.draw_geometries([voxel_grid],height=800, width=800)

# 09. Create triangle mesh out of the points that are left
not used

In [64]:
# Ensure normals are oriented consistently
filtered_pcd.orient_normals_consistent_tangent_plane(100)

# Poisson surface reconstruction
mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(filtered_pcd, depth=5)

# Crop the mesh to remove unwanted parts
bbox = filtered_pcd.get_axis_aligned_bounding_box()
mesh = mesh.crop(bbox)

# Visualize the reconstructed mesh
o3d.visualization.draw_geometries([mesh], 
                                  mesh_show_wireframe=True,
                                  mesh_show_back_face=True
                                  )

# 09.b Extract edges


edges between 2 neighbours - short version 

In [70]:
# Visualize filtered point cloud
#o3d.visualization.draw_geometries([filtered_pcd], height=800, width=800)
voxel_size = 0.00001

def calculate_angle_between_normals(normal1, normal2):
    """Calculate the angle between two normals in degrees."""
    dot_product = np.dot(normal1, normal2)
    dot_product = np.clip(dot_product, -1.0, 1.0)  # Clip to the range [-1, 1]
    angle = np.arccos(dot_product) * (180.0 / np.pi)
    return angle

# Convert points and normals to numpy arrays
points = np.asarray(filtered_pcd.points)
normals = np.asarray(filtered_pcd.normals)

# KDTree for neighbor search
kdtree = o3d.geometry.KDTreeFlann(filtered_pcd)

# Define the angle threshold
angle_threshold = 45.0  # in degrees

# List to store indices of points to keep
indices_to_keep = []

# Iterate over all points
for i in range(len(points)):
    [_, idx, _] = kdtree.search_radius_vector_3d(filtered_pcd.points[i], voxel_size * 2)
    keep_point = False
    for j in idx:
        if i != j:
            angle = calculate_angle_between_normals(normals[i], normals[j])
            if angle > angle_threshold:
                keep_point = True
                break
    if keep_point:
        indices_to_keep.append(i)

# Select the filtered points
filtered_points = filtered_pcd.select_by_index(indices_to_keep)

# Visualize the filtered points
o3d.visualization.draw_geometries([filtered_points], height=800, width=800)

5 neighbours in any direction - long version

In [69]:
import open3d as o3d
import numpy as np

def calculate_angle_between_normals(normal1, normal2):
    """Calculate the angle between two normals in degrees."""
    dot_product = np.dot(normal1, normal2)
    dot_product = np.clip(dot_product, -1.0, 1.0)  # Clip to the range [-1, 1]
    angle = np.arccos(dot_product) * (180.0 / np.pi)
    return angle

# Load the point cloud
# pcd = o3d.io.read_point_cloud("point_cloud.ply")

# # Downsample the point cloud (optional)
# voxel_size = 0.0001
# filtered_pcd = pcd.voxel_down_sample(voxel_size=voxel_size)

# Estimate normals
#filtered_pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

# Convert points and normals to numpy arrays
points = np.asarray(filtered_pcd.points)
normals = np.asarray(filtered_pcd.normals)

# KDTree for neighbor search
kdtree = o3d.geometry.KDTreeFlann(filtered_pcd)

# Define the angle threshold
angle_threshold = 45.0  # in degrees

# List to store indices of points to keep
indices_to_keep = []

# Iterate over all points
for i in range(len(points)):
    [_, idx, _] = kdtree.search_radius_vector_3d(filtered_pcd.points[i], voxel_size * 2)
    keep_point = False
    for j in idx:
        if i != j:
            angle = calculate_angle_between_normals(normals[i], normals[j])
            if angle > angle_threshold:
                for k in idx:
                    if k != i and k != j:
                        opposite_angle = calculate_angle_between_normals(normals[i], normals[k])
                        if opposite_angle > angle_threshold:
                            keep_point = True
                            break
                if keep_point:
                    break
    if keep_point:
        indices_to_keep.append(i)

# Select the filtered points
filtered_points = filtered_pcd.select_by_index(indices_to_keep)

# Visualize the filtered points
o3d.visualization.draw_geometries([filtered_points], height=800, width=800)

Try to optimize for better performance

# 10.a Try create a convex hull

In [74]:
# Downsample 
voxel_size = 0.0001  # Adjust this value based on your data
filtered_points_down = filtered_points.voxel_down_sample(voxel_size=voxel_size)


In [75]:
hull, _ = filtered_points_down.compute_convex_hull()
hull_ls = o3d.geometry.LineSet.create_from_triangle_mesh(hull)
hull_ls.paint_uniform_color([1, 0, 0])
o3d.visualization.draw_geometries([filtered_points_down, hull_ls])

# 10.b Try to bound the box around the object

--- Does not look satisfactionary

In [15]:
# obb = filtered_pcd.get_oriented_bounding_box()
# obb.color = [0, 1, 0]
# o3d.visualization.draw_geometries([filtered_pcd, obb])

# 10.c Use ball pivoting to estimate mesh

In [ ]:
cras 

In [16]:
# Downsample to make compuatations feasible
down_pcd = filtered_pcd.voxel_down_sample(voxel_size=0.00001)
down_pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
o3d.visualization.draw_geometries([down_pcd], 
                                   point_show_normal=True,
                                   mesh_show_wireframe=True, 
                                   mesh_show_back_face=True
                                   )

print(f"Number of points in non plane: {len(down_pcd.points)}")

In [ ]:
voxel_size = 0.00001
print(f"Voxel size: {voxel_size}")
radii = [8 * voxel_size, 4*voxel_size, 16* voxel_size]#, 0.000002]
rec_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
    down_pcd, o3d.utility.DoubleVector(radii)
)

In [21]:
# Visualize
o3d.visualization.draw_geometries([rec_mesh], 
                                  mesh_show_wireframe=True,
                                  mesh_show_back_face=True
                                  )